# All settings will be given to the channel 1 output, in the Direct out DAC configuration

In [1]:
%matplotlib widget 

In [2]:
import numpy as np
import pandas as pd
import time
import matplotlib
import matplotlib.pyplot as plt
import pyvisa as visa
from matplotlib import animation
from time import sleep
from matplotlib.animation import FuncAnimation
#Importing the functions from the module and the pulses dictionaries
from Sweeping import *
from Dictionaries import *
from PulseFiles import *
from Instrument import *
import pyqtgraph as pg 
import nidaqmx

# Visa interface for the M8190A instrument:

In [3]:
M8190A = VisaR(AWG_Settings4,60000)

In [4]:
Initialization(M8190A,AWG_Settings4)

Instruments Sampling Frecuency set to 5.00000000000000E+08Hz
Instruments Direct Out DAC Output route Voltage set to 7.00000000000000E-01deciVolts
AWG set to TRIGGERED Mode
Trigger In threshold value set to 1.00000000000000E+00V


In [5]:
M8190A.query('*IDN?')

'Agilent Technologies,M8190A,MY57701776,5.6.0.0-2'

In [26]:
M8190A.write('*RST')

5

In [5]:
M8190A.query('FREQ:RAST?')

'5.00000000000000E+08'

In [18]:
M8190A.query(':DAC:VOLT:AMPL?')

'7.00000000000000E-01'

In [5]:
M8190A.query('SYST:ERR?')

'0,"No error"'

In [4]:
print(M8190A)

TCPIPInstrument at TCPIP0::ibn3-036.ibn-net.kfa-juelich.de::hislip0::INSTR


# Loading and triggering a single waveform:

In [6]:
M8190A.query('ARM:TRIG:LEV?')

'1.00000000000000E+00'

In [6]:
M8190A.write('ABOR')

5

In [8]:
test_run0 = Sequence_Pulse_List(PList_secondsA,PList_secondsB,10,0,15,7500000,M8190A,AWG_Settings4,1)

Current Segment Catalogue is 1,7500000  [(segment id, Segment size)]
Current Segment Catalogue is 1,7500000,2,7500000  [(segment id, Segment size)]
Sequence advancement method is COND
Sequence loaded with the following segment data "1,1,0,1,0,7499999,2,1,0,1,0,7499999"


In [4]:
test_run = Triggered_Sequence_Setup(PList_secondsA,PList_secondsB,10,0,15,7500000,M8190A,AWG_Settings4,1)

Current Segment Catalogue is 1,7500000  [(segment id, Segment size)]
Current Segment Catalogue is 1,7500000,2,7500000  [(segment id, Segment size)]
Sequence advancement method is COND
Sequence loaded with the following segment data "1,1,0,1,0,7499999,2,1,0,1,0,7499999"


In [10]:
test_run[0]['Y1']*AWG_Settings4['Voltage Amplitude']

0            0.0
1          500.0
2          500.0
3          500.0
4          500.0
           ...  
7499995      0.0
7499996      0.0
7499997      0.0
7499998      0.0
7499999      0.0
Name: Y1, Length: 7500000, dtype: float64

In [11]:
test_run[1]['Y1']*AWG_Settings4['Voltage Amplitude']

0            0.0
1          500.0
2          500.0
3          500.0
4          500.0
           ...  
7499995      0.0
7499996      0.0
7499997      0.0
7499998      0.0
7499999      0.0
Name: Y1, Length: 7500000, dtype: float64

In [5]:
fig,ax= plt.subplots(2)

ax[0].grid(True)
ax[1].grid(True)
fig.suptitle('Pulses Loaded into the AWG')


ax[0].plot(test_run[2],test_run[0]['Y1']*AWG_Settings4['Voltage Amplitude'],label= 'Pulse A',color='blue')
ax[1].plot(test_run[2],test_run[1]['Y1']*AWG_Settings4['Voltage Amplitude'],label= 'Pulse B',color='red')
ax[0].set_ylabel('V  [mV]')
ax[0].set_xlabel('t  [ms]')
ax[1].set_ylabel('V  [mV]')
ax[1].set_xlabel('t  [ms]')

#ax.set_title('Pulses Loaded into the AWG')

fig.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.show()

#fig.savefig('Pulse_scheme_15ms_7500000',optimize=True,bbox_inches='tight')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
Trigger_Pulse('DAQBNC1','ao1',1.5,4)

Triggering Pulse Stoped


In [12]:
25*1e-3

0.025

In [22]:
DAQ_Settings1['Sampling Frequency'] * 25 *1e-3

10000.0

In [6]:
Trigger_Pulse('DAQBNC1','ao1',1.5,4)

Triggering Pulse Stoped


In [20]:
dataM, timeM = DAQ_Measuring0(DAQ_Settings1,1000,10,M8190A)

fig,ax= plt.subplots()
ax.grid(True)

plt.plot(timeM,dataM*1000,label= '{b} samples'.format(b=len(dataM)),color='orange')

ax.set_ylabel('V  [mV]')
ax.set_xlabel('t  [s]')
ax.set_title('Voltage measured with DAQ, Direct out Channel')
#ax.legend(loc=0)

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.show()
#fig.savefig('DAQ measurement noise0',bbox_inches='tight')

Triggering Pulse Stoped


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\a.hernandez.de.la.ve\Anaconda3\lib\site-packages\nidaqmx\task.py:102: ResourceWarning: Task of name "_unnamedTask<6>" was not explicitly closed before it was destructed. Resources on the task device may still be reserved.
  warnings.warn(
C:\Users\a.hernandez.de.la.ve\Anaconda3\lib\site-packages\nidaqmx\task.py:102: ResourceWarning: Task of name "_unnamedTask<7>" was not explicitly closed before it was destructed. Resources on the task device may still be reserved.
  warnings.warn(


# Different behaviour when initializing the AWG outside of the DAQ_measurement function (or inside, but with a time.sleep() right after it) and when initializing the AWG inside the function. Initializing with a delay gives "proper" measurement scheme (I still have to check whether it is getting the correct time scale). This independant of the delay time present when loading a waveform to the awg for the first time after reseting.

In [16]:
dataM.size

10000

In [22]:
a = 0# * e-1

In [18]:
M8190A.write(':DAC1:VOLT:OFFS 0.00{value}'.format(value = a ))

23

Pump = 40mV

ProbeA = 20mV

ProbeB = - 20mV

In [23]:
dataMs, timeMs = DAQ_Measuring0ms(DAQ_Settings1,400000,32,M8190A)


fig,ax= plt.subplots()

ax.grid(True)
#plt.plot(tom,data_off,label= 'AWG off')
plt.plot(timeMs,dataMs*1000,label= '{b} samples'.format(b=len(dataMs)),color='orange')

ax.set_ylabel('V  [mV]')
ax.set_xlabel('t  [ms]')
ax.set_title('Voltage measured with DAQ, Direct out Channel DAC, Voltage Offset = {value}mV '.format(value = a*0.1))
ax.legend(loc=0)
#plt.legend(bbox_to_anchor=(2.05, 1), loc='upper left', borderaxespad=0.)
plt.show()
#fig.savefig('DAQ measurement offset {value}mV '.format(value = a ),bbox_inches='tight')

Triggering Pulse Stoped


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Mergin Sweeping loading function with DAQ measuring

In [6]:
def Sequence_Loader_List_Files0(PulseList1,PulseList2,P,t,N,start,stop,AWG):

    """ Given two pulse schemes lists, this functions iterates over them from start to stop.

        This function firts creates the corresponding pulse sequence data given the PulseLists using the Sweep_iteration_csv function
        
    """

    #SegmentA of the sequence
    Loc1,DF1,timm = Sweep_Iteration_CSV_List(PulseList1,P,t,N,start,stop,AWG,1)

    #SegmentB of the Sequence
    Loc2,DF2,timm = Sweep_Iteration_CSV_List(PulseList2,P,t,N,start,stop,AWG,0)

    return Loc1, Loc2, DF1,DF2,timm

In [25]:
def Sequence_Loader_File_DAQ(instrument,DAQ_settings,sampling_rate,playingtime,fileA,fileB):
    
    """ This function loads the csv data files from the Location dictionaries into the instrument as a sequence. FOR SWEEPING

    LocationA is a dictionary, whose elements are the file paths to the csv files that are going to be loaded as SegmentA into the sequence.
    LocationB is a dictionary, whose elemnts re the filepaths to the csv files that are going to be loaded as SegmentB into the sequence
    It uses the Sequence_File function to load the csv files to the instrument.
 
    """
    measurement_data = {}

    for i,j in zip(fileA, fileB):
        Sequence_File(instrument,fileA[i],fileB[j],1)
        instrument.query('*OPC?')
        measurement_data['data result for step {a}and{b}'.format(a =i, b = j)]= DAQ_Measuring(DAQ_settings,sampling_rate,playingtime,instrument)

    return measurement_data
    instrument.write('ABOR')

In [7]:
sweeping_run = Sequence_Loader_List_Files0(PList_secondsA,PList_secondsB,10,15,7500000,0,4,AWG_Settings4)

desmenuzar

In [26]:
Sequence_Loader_File_DAQ(M8190A,DAQ_Settings1,400000,32,sweeping_run[0],sweeping_run[1])

Sequence advancement method is COND
Sequence loaded with the following segment data "1,1,0,1,0,7499999,2,1,0,1,0,7499999"


DaqError: Some or all of the samples requested have not yet been acquired.
To wait for the samples to become available use a longer read timeout or read later in your program. To make the samples available sooner, increase the sample rate. If your task uses a start trigger,  make sure that your start trigger is configured correctly. It is also possible that you configured the task for external timing, and no clock was supplied. If this is the case, supply an external clock.
Property: DAQmx_Read_RelativeTo
Corresponding Value: DAQmx_Val_CurrReadPos
Property: DAQmx_Read_Offset
Corresponding Value: 0

Task Name: _unnamedTask<2>

Status Code: -200284

In [21]:
sweeping_run[0]

{'Pulse File Location, at step 0': 'D:\\Alejandro\\Pulses\\Dict\\S4\\SegmentA_7500000_0.csv',
 'Pulse File Location, at step 1': 'D:\\Alejandro\\Pulses\\Dict\\S4\\SegmentA_7500000_1.csv',
 'Pulse File Location, at step 2': 'D:\\Alejandro\\Pulses\\Dict\\S4\\SegmentA_7500000_2.csv',
 'Pulse File Location, at step 3': 'D:\\Alejandro\\Pulses\\Dict\\S4\\SegmentA_7500000_3.csv',
 'Pulse File Location, at step 4': 'D:\\Alejandro\\Pulses\\Dict\\S4\\SegmentA_7500000_4.csv'}

In [12]:
sweeping_run[1]

{'Pulse File Location, at step 0': 'D:\\Alejandro\\Pulses\\Dict\\S4\\SegmentB_7500000_0.csv',
 'Pulse File Location, at step 1': 'D:\\Alejandro\\Pulses\\Dict\\S4\\SegmentB_7500000_1.csv',
 'Pulse File Location, at step 2': 'D:\\Alejandro\\Pulses\\Dict\\S4\\SegmentB_7500000_2.csv',
 'Pulse File Location, at step 3': 'D:\\Alejandro\\Pulses\\Dict\\S4\\SegmentB_7500000_3.csv',
 'Pulse File Location, at step 4': 'D:\\Alejandro\\Pulses\\Dict\\S4\\SegmentB_7500000_4.csv'}

In [15]:
sweeping_run[2]

{'SegmentA_7500000_0':                Y1  SyncMarker1  SampleMarker1
 0        0.000000            1              1
 1        0.057143            1              1
 2        0.057143            1              1
 3        0.057143            1              1
 4        0.057143            1              1
 ...           ...          ...            ...
 7499995  0.000000            1              1
 7499996  0.000000            1              1
 7499997  0.000000            1              1
 7499998  0.000000            1              1
 7499999  0.000000            1              1
 
 [7500000 rows x 3 columns],
 'SegmentA_7500000_1':                Y1  SyncMarker1  SampleMarker1
 0        0.000000            1              1
 1        0.057143            1              1
 2        0.057143            1              1
 3        0.057143            1              1
 4        0.057143            1              1
 ...           ...          ...            ...
 7499995  0.000000            1 

In [16]:
sweeping_run[3]

{'SegmentB_7500000_0':                Y1  SyncMarker1  SampleMarker1
 0        0.000000            0              0
 1        0.057143            0              0
 2        0.057143            0              0
 3        0.057143            0              0
 4        0.057143            0              0
 ...           ...          ...            ...
 7499995  0.000000            0              0
 7499996  0.000000            0              0
 7499997  0.000000            0              0
 7499998  0.000000            0              0
 7499999  0.000000            0              0
 
 [7500000 rows x 3 columns],
 'SegmentB_7500000_1':                Y1  SyncMarker1  SampleMarker1
 0        0.000000            0              0
 1        0.057143            0              0
 2        0.057143            0              0
 3        0.057143            0              0
 4        0.057143            0              0
 ...           ...          ...            ...
 7499995  0.000000            0 

In [18]:
sweeping_run[4].size

7500000

In [ ]:
fig,ax= plt.subplots()

ax.grid(True)
plt.plot(tom,obo[0],label= 'Starting Step 0')
plt.plot(tom,obo[1],label= '1')
plt.plot(tom,obo[2],label= '2')
plt.plot(tom,obo[3],label= '3')
plt.plot(tom,obo[4],label= '4')
plt.plot(tom,obo[5],label= '5')
plt.plot(tom,obo[6],label= '6')
plt.plot(tom,obo[7],label= '7')
plt.plot(tom,obo[8],label= '8')
plt.plot(tom,obo[9],label= '9')
plt.plot(tom,obo[10],label= 'Stopping Step 10')
ax.set_ylabel('V  [mV]')
ax.set_xlabel('t  [ms]')
ax.set_title(' Complete Sweeping Steps ')
ax.legend(loc=0)
plt.show()
#fig.savefig('Pulse_sweep_t=20',optimize=True,bbox_inches='tight')

In [19]:
def DAQ_Measuring(DAQ_settings,sr,playingtime,instrument):
    """This function starts sets up the DAQ box in order to collect data for a time duration given by "playing time"
      It then uses the DAQ box to trigger the AWG into playing a waveform.

      playingtime should be in seconds.
      triggerinvoltage should be in volts.
    """
    instrument.write('INIT:IMM')
    time.sleep(2)


    #Calculating the number of samples given the samplig frecuency and playing time
    samples = int(sr * playingtime) 
    measuring_time = np.linspace(0,playingtime,samples)

    #setting the tasks
    measuring_task = nidaqmx.Task()
    trig_task =  nidaqmx.Task()
   


    #Channels Configuration
    measuring_task.ai_channels.add_ai_voltage_chan("{a}/{b}".format(a = DAQ_settings['DAQ Name'], b = DAQ_settings['Analog Channel Input']),min_val=DAQ_settings['Minimum Voltage'],max_val= DAQ_settings['Maximum Voltage'])
    trig_task.ao_channels.add_ao_voltage_chan('{a}/{b}'.format(a = DAQ_settings['DAQ Name'], b = DAQ_settings['Analog Channel Output']),'triggering',-4,4)

    #Sampling configuration measuring channel
    measuring_task.timing.cfg_samp_clk_timing(sr, active_edge=Edge.RISING, sample_mode=AcquisitionType.FINITE, samps_per_chan=samples)
    #trig_task.timing.cfg_samp_clk_timing(DAQ_settings['Sampling Frequency'], samps_per_chan=samples)
    #source = "measuring_task/SampleClock"

    trig_task.start()
    measuring_task.start()
    

    
    trig_task.write(1.5)
    #time.sleep(3)
    data = np.array(measuring_task.read(samples))


    
    #time.sleep(3)
    print('Triggering Pulse Stoped')
    trig_task.write(0)

    
    
    

    trig_task.stop()
    measuring_task.stop()

    measuring_task.close()
    trig_task.close()


    instrument.write('ABOR')

    return data, measuring_time

In [12]:
M8190A.query('OUTP:ROUT:SEL?')

'DAC'

In [11]:
M8190A.write('OUTP:ROUT:SEL DAC')

18

In [17]:
M8190A.write('INIT:IMM')

9

voltask =  nidaqmx.Task()

voltask.ao_channels.add_ao_voltage_chan("DAQBNC1/ao1",min_val=-3,max_val=3)





print('Task started')
voltask.start()

time.sleep(10)

print('Writting!')

time.sleep(4)
voltask.write(3)
time.sleep(7) 

print('Lowering Voltage!')
time.sleep(4)

voltask.write(0)
#
voltask.stop()
#
voltask.close()